In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df_alcohol = pd.read_csv('../data/processed/alcohol_data.csv')
df_bp = pd.read_csv('../data/processed/blood_pressure_data.csv')
df_country_code = pd.read_csv('../data/processed/country_codes.csv')

In [4]:
df_country_code

,Country,Country_Code
0,Afghanistan,AFG
1,Afghanistan,AFG
2,Afghanistan,AFG
3,Algeria,DZA
4,Andorra,AND
...,...,...
47784,United States of America,USA
47785,United States of America,USA
47786,Austria,AUT
47787,United States of America,USA


In [5]:
df_alcohol = df_alcohol.set_index('Year')
df_bp = df_bp.set_index('Year')

In [6]:
year = 1980

df_a_single_year = pd.DataFrame({'alcohol': df_alcohol.loc[year]})
df_bp_single_year = pd.DataFrame({'bp': df_bp.loc[year]}) 

In [7]:
df_a_single_year = df_a_single_year.reset_index()
df_a_single_year.columns = ['Country', 'Alcohol']

df_bp_single_year = df_bp_single_year.reset_index()
df_bp_single_year.columns = ['Country', 'BP']

In [8]:
df_a_single_year = pd.merge(df_a_single_year, df_country_code, on='Country', how='left')

In [9]:
df_merge = pd.merge(df_a_single_year, df_bp_single_year, on='Country')

In [10]:
df_merge = df_merge.dropna()

In [11]:
df_cont = pd.read_csv('../data/raw/continent_data.csv')

In [12]:
df_cont = df_cont[['alpha-3', 'sub-region', 'region']]

In [13]:
df_cont.columns = ['Country_Code', 'sub-region', 'region']

In [23]:
list(df_cont.region.unique())

['Asia', 'Europe', 'Africa', 'Oceania', 'Americas', nan]

In [15]:
df_cont.columns

Index(['Country_Code', 'sub-region', 'region'], dtype='object')

In [16]:
def get_continent(row):
    if row.region in ['Africa', 'Asia', 'Europe', 'Oceania']:
        return row.region
    else:
        if row['sub-region'] == 'Northern America':
            return 'North America'
        else:
            return 'South America'

In [17]:
df_merge.iloc[0]

Country         Albania
Alcohol            1.79
Country_Code        ALB
BP                 32.5
Name: 270, dtype: object

In [18]:
df_cont['continent'] = df_cont.apply(get_continent, axis=1)

In [19]:
df_cont.continent.value_counts()

Africa           60
South America    53
Asia             51
Europe           51
Oceania          29
North America     5
Name: continent, dtype: int64

In [20]:
continent_dictionary = {"Europe": ['Southern Europe', 'Northern Europe', 'Southern Europe', 'Eastern Europe'],
'Asia':['Western Asia', 'South-eastern Asia', 'Southern Asia', 'Eastern Asia', 'Central Asia'],
'Africa': ['Sub-Saharan Africa', 'Northern Africa'],
'Oceania': ['Australia and New Zealand', 'Micronesia', 'Polynesia'],
'North America': ['Northern America'],
'South America': ['Latin Amaerica and the Caribbean']}

In [21]:
df_merge = pd.merge(df_merge, df_cont, how='left', on='Country_Code')

In [22]:
fig = px.scatter(df_merge, x="Alcohol", y="BP",
                 hover_data=['Country'], color='continent')
fig.show()